In [1]:
%pylab inline
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


In [14]:
def one_hot(my_df, col, drop=True):
    for i in my_df[col].unique():
        my_df[col + "_" + str(i)] = my_df[col].apply(lambda x: 1 if x == i else 0)
    if(drop):
        my_df.drop(col, inplace=True, axis=1)

In [15]:
train = pd.read_csv('data/train.csv')

cats = [c for c,d in zip(train.columns, train.dtypes) if str(d) == 'object']

for i in cats:
    one_hot(train, i)


In [18]:
train = train.dropna()

In [19]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression

clf = LinearRegression()
cross_val_score(clf, train.drop("SalePrice", axis=1), train["SalePrice"])

array([ 0.85457983, -1.25603044,  0.63885102])